In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import pandas as pd
import pprint

pd.options.display.float_format = '{:20,.4f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

# open file
x_rates = pd.read_csv("euro-daily-hist_1999_2020.csv")

# strip brackets and trailing space from country names
# replace remaining space with underscore
# lower case all column names
x_rates.columns = x_rates.columns.str.replace("[","").str.replace("]","")
x_rates.columns = x_rates.columns.str.rstrip()
x_rates.columns = x_rates.columns.str.replace(" ","_")
x_rates.columns = x_rates.columns.str.lower()

# rename columns
x_rates.rename(columns={"period\\unit:":"date",
                        "chinese_yuan_renminbi":"chinese_yuan",
                        "uk_pound_sterling":"uk_pound"}, inplace=True)

# convert datetime
x_rates["date"] = pd.to_datetime(x_rates["date"])

# resort and reindex
x_rates.sort_values("date", inplace=True)
x_rates.reset_index(drop=True, inplace=True)

# convert hyphens in currency columns to NaN
import numpy as np
x_rates = x_rates.replace("-", np.nan)

# convert exchange rate values to float
x_rates.iloc[:,1:] = x_rates.iloc[:,1:].astype(float)

mini = x_rates.iloc[:5000]
display(mini.head(2))
display(mini.tail(2))

/var/folders/bz/5cj1tzbj5xn319rpqrw6gpmh0000gn/T/ipykernel_9008/758582820.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x_rates.columns = x_rates.columns.str.replace("[","").str.replace("]","")


,date,australian_dollar,bulgarian_lev,brazilian_real,canadian_dollar,swiss_franc,chinese_yuan,cypriot_pound,czech_koruna,danish_krone,estonian_kroon,uk_pound,greek_drachma,hong_kong_dollar,croatian_kuna,hungarian_forint,indonesian_rupiah,israeli_shekel,indian_rupee,iceland_krona,japanese_yen,korean_won,lithuanian_litas,latvian_lats,maltese_lira,mexican_peso,malaysian_ringgit,norwegian_krone,new_zealand_dollar,philippine_peso,polish_zloty,romanian_leu,russian_rouble,swedish_krona,singapore_dollar,slovenian_tolar,slovak_koruna,thai_baht,turkish_lira,us_dollar,south_african_rand
0,1999-01-04,1.9100,NaN,NaN,1.8004,1.6168,NaN,0.5823,35.1070,7.4501,15.6466,0.7111,327.1500,9.1332,NaN,251.4800,"9,433.6100",NaN,NaN,81.4800,133.7300,"1,398.5900",4.7170,0.6668,0.4432,11.6446,4.4798,8.8550,2.2229,45.5100,4.0712,1.3111,25.2875,9.4696,1.9554,189.0450,42.9910,42.6799,0.3723,1.1789,6.9358
1,1999-01-05,1.8944,NaN,NaN,1.7965,1.6123,NaN,0.5823,34.9170,7.4495,15.6466,0.7122,324.7000,9.1341,NaN,250.8000,"9,314.5100",NaN,NaN,81.5300,130.9600,"1,373.0100",4.7174,0.6657,0.4432,11.5960,4.4805,8.7745,2.2011,44.7450,4.0245,1.3168,26.5876,9.4025,1.9655,188.7750,42.8480,42.5048,0.3728,1.1790,6.7975


,date,australian_dollar,bulgarian_lev,brazilian_real,canadian_dollar,swiss_franc,chinese_yuan,cypriot_pound,czech_koruna,danish_krone,estonian_kroon,uk_pound,greek_drachma,hong_kong_dollar,croatian_kuna,hungarian_forint,indonesian_rupiah,israeli_shekel,indian_rupee,iceland_krona,japanese_yen,korean_won,lithuanian_litas,latvian_lats,maltese_lira,mexican_peso,malaysian_ringgit,norwegian_krone,new_zealand_dollar,philippine_peso,polish_zloty,romanian_leu,russian_rouble,swedish_krona,singapore_dollar,slovenian_tolar,slovak_koruna,thai_baht,turkish_lira,us_dollar,south_african_rand
4998,2018-04-13,1.5801,1.9558,4.1979,1.5482,1.1854,7.7363,NaN,25.3070,7.4467,NaN,0.8640,NaN,9.6687,7.4165,311.1300,"16,939.5700",4.3316,80.3160,121.6000,132.6400,"1,316.2600",NaN,NaN,NaN,22.3162,4.7714,9.5643,1.6703,63.9690,4.1763,4.6603,76.2186,10.3798,1.6158,NaN,NaN,38.3670,5.0411,1.2317,14.8457
4999,2018-04-16,1.5928,1.9558,4.2300,1.5596,1.1878,7.7726,NaN,25.2650,7.4473,NaN,0.8647,NaN,9.7104,7.4128,310.3000,"17,039.9800",4.3327,81.0175,122.0000,132.7700,"1,328.3300",NaN,NaN,NaN,22.3399,4.8057,9.5950,1.6853,64.3870,4.1627,4.6508,76.9420,10.4045,1.6221,NaN,NaN,38.6440,5.0816,1.2370,14.9467


In [3]:
# create a list of data frame names
# not sure this is neccessary but trying df = df_list.index(currency) in the framer function doesn't seem to work
# create a list of data frames for each currency with log rate of the exchange rate, 30 day rolling mean, and year
col_names = []
df_list = []
df_dict = {}
for column in mini.columns[1:]:
    df_name = column
    col_names.append(df_name)
    df = mini[["date", column]].copy()
    df = df[df[column].notna()]
    df["log_rate"] = np.log(df.iloc[:,1]/df.iloc[:,1].shift()) # getting the log of the exchange rate # double check this is the correct way to get log
    df["rolling_mean_30"] = df[column].rolling(30).mean()
    df["year"] = df["date"].dt.year
#     df_dict[column] = df
    df_list.append(df)

In [4]:
# functions to build annual volatility given string of currency name
# function to assign dataframe to variable name
# i could include this in volatizer, but for now keep separate because these are frames of all the original data
def framer(currency):
    index = col_names.index(currency)
    df = df_list[index] # this is a dataframe containing a single currency and the columns built in cell 3
    return df

# function to create df with year and annual volatility for every row # i think i could use aggregation here but don't know how
def volatizer(currency):
    all_the_years = [currency[currency['year'] == y] for y in currency['year'].unique()] # list of dataframes for each year
    c_name = currency.columns[1]
    df_dict = {}
    for frame in all_the_years:
        year_name = frame.iat[0,4] # the year for each df, becomes the "year" cell for annual volatility df
        annual_volatility = frame["log_rate"].std()*253**.5 # volatility measured by standard deviation * 253 trading days per year raised to the 0.5 power
        df_dict[year_name] = annual_volatility
    df = pd.DataFrame.from_dict(df_dict, orient="index", columns=[c_name+"_annual_vol"]) # indexing on year, not sure if this is cool
    return df
        
brazilian_real = framer("brazilian_real")
br_vol = volatizer(brazilian_real)
# now i need to run this on all 40 df's and get a new df with all of them joined
# then build an hbar chart with the eight from above
# maybe two hbar charts for cell phones?

In [5]:
display(br_vol)

,brazilian_real_annual_vol
2000,0.2621
2001,0.2217
2002,0.3007
2003,0.2003
2004,0.1244
2005,0.1670
2006,0.1669
2007,0.1549
2008,0.3049
2009,0.1553


In [10]:
# use string of currency to get index position and assign it to variable name
index = col_names.index("chinese_yuan")
print(index)
chinese_yuan = df_list[5]

# function to create df for each year in currency df
def split_years(df):
    return [df[df['year'] == y] for y in df['year'].unique()]

all_the_years = split_years(chinese_yuan)
# see if I can make split_years split and heads


# # determine annual volatility for euro to currency
# for frame in all_the_years:
#     year_name = frame.iat[0,4]
#     annual_volatility = frame["log_rate"].std()*253**.5 # volatility measured by standard deviation * 253 trading days per year
#     print("The Euro to Chinese yuan volatility for", year_name, "is" ,annual_volatility)

5


In [11]:
display(all_the_years[0])

,date,chinese_yuan,log_rate,rolling_mean_30,year
268,2000-01-13,8.5054,NaN,NaN,2000
269,2000-01-14,8.4632,-0.0050,NaN,2000
270,2000-01-17,8.3548,-0.0129,NaN,2000
271,2000-01-18,8.3540,-0.0001,NaN,2000
272,2000-01-19,8.3639,0.0012,NaN,2000
273,2000-01-20,8.3507,-0.0016,NaN,2000
274,2000-01-21,8.3573,0.0008,NaN,2000
275,2000-01-24,8.2993,-0.0070,NaN,2000
276,2000-01-25,8.2836,-0.0019,NaN,2000
277,2000-01-26,8.2927,0.0011,NaN,2000
